In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np
import pandas as pd 
import seaborn as sb
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
print(os.listdir("../input/garbage-classification/garbage classification/Garbage classification/"))
['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']
Preparing the data and Visualization

trash = pd.read_csv("/kaggle/input/garbage-classification/zero-indexed-files.txt")


    
# print(filename ,end="\n")
#Create new dataframe with two columns




In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sb
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
print(os.listdir("../input/garbage-classification/garbage classification/Garbage classification/"))
['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']
Preparing the data and Visualization

trash = pd.read_csv("/kaggle/input/garbage-classification/zero-indexed-files.txt")


In [ ]:

category = []
# index = []
filename = []
with open("../input/garbage-classification/zero-indexed-files.txt", "r") as file:
    for line in file:     
#      5>4?sd:dsds
        # Category of the image (label) e.g. glass
        cat = ''.join([i for i in line.split('.')[0] if not i.isdigit()])
        category.append(cat)
        
        
       #Image name e.g. glass/glass123.jpg
        filename.append(cat+"/"+line.split(' ')[0])
        
        
        #numerical value for category e.g. 0 for glass
#         index.append((line.split(' ')[1]).strip()

In [ ]:
df = pd.DataFrame({
    'filename':filename,
    'category': category,
#     'index': index
})

In [ ]:


plt.title('Count plot')
sb.countplot(x = df['category'], data=df)
#Index 0: glass, Index 1: paper, Index 2: cardboard, Index 3: plastic, Index 4: metal, Index 5: other
<AxesSubplot:title={'center':'Count plot'}, xlabel='category', ylabel='count'>

sample = random.choice(filename)
folder = ''.join([i for i in sample.split('.')[0] if not i.isdigit()])
    
image = load_img("../input/garbage-classification/garbage classification/Garbage classification/"+sample)
plt.title(folder.split('/')[0])
plt.imshow(image)

In [ ]:


FAST_RUN = False
IMAGE_WIDTH=300
IMAGE_HEIGHT=300
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

#padding='same' returns output whose value can be computed by applying the filter to all input elements. 
#Border elements are computed using zero padding. 
#The output may be same or smaller than the input depending on the stride option.

model = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(300, 300, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),

    Flatten(),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(6, activation='softmax') #6 because we have 6 categories
])

#Computes the crossentropy loss between the labels and predictions.
#acc metric to be evaluated by the model during training and testing
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#stop training when a monitored metric has stopped improving.
#patience: number of epochs with no improvement after which training will be stopped.
earlystop = EarlyStopping(patience=10)


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', #quantity to be monitored (in this case total loss)
                                            patience=2, #number of epochs with no improvement after which learning rate will be reduced
                                            verbose=1, #0: quiet, 1: update messages
                                            factor=0.5, #factor by which the learning rate will be reduced
                                            min_lr=0.00001) #lower bound on the learning rate.

In [ ]:
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
# df['index'] = df['category'] #.replace({0: 'glass', 1: 'paper', 2: 'cardboard', 3: 'plastic', 4: 'metal', 5: 'trash'}) 
# df.drop('index', inplace=True, axis=1)
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)#42 is the seed of randomness

#Incase a column was part of an index
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

sb.countplot(x = train_df['category'])

In [ ]:
sb.countplot(x = validate_df['category'])

In [ ]:
#.shape[0]: gives number of rows
#batch_size: number of training examples
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15, #range for random rotations
    rescale=1./255, #rescaling factor to transform every pixel value from range [0,255] -> [0,1]
    shear_range=0.1, #shear intensity, the image will be distorted along an axis
    zoom_range=0.2, #zoom in an image
    horizontal_flip=True, #randomly flip inputs horizontally
    width_shift_range=0.1, #shift horizontally(left or right)
    height_shift_range=0.1 #shift vertically(up or down)
)

#Takes the dataframe and the path to a directory and generates batches.
#The generated batches contain augmented/normalized data.
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "../input/garbage-classification/garbage classification/Garbage classification", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',#the type of label arrays that are returned (in this case categorical)
    batch_size=batch_size
)
# train_generator = train_datagen.flow_from_directory(
#     "../input/garbage-classification/garbage classification/Garbage classification", 
#     target_size=(300, 300),
#     batch_size=16,
#     class_mode='categorical',
#     subset='training',
#     seed=0
# )

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255) #rescaling factor to transform every pixel value from range [0,255] -> [0,1]
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "../input/garbage-classification/garbage classification/Garbage classification", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True) #n is number of images returned
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "../input/garbage-classification/garbage classification/Garbage classification", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical'
)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1) #(rows,columns,index)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

In [ ]:
model.save("model_model.h5")
model.save_weights("weight_model.h5")

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 12))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='red')
plt.plot(val_acc, label='Validation Accuracy', color='blue')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='red')
plt.plot(val_loss, label='Validation Loss', color='blue')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
test_x, test_y = validation_generator.__getitem__(1) #I think gets index 1 from dataframe (need to make sure)

labels = (train_generator.class_indices) #the mapping from class names to class indices
labels = dict((v,k) for k,v in labels.items()) #creates a dictionary by iterating through the indices

preds = model.predict(test_x)

plt.figure(figsize=(16, 16))
for i in range(15):
    plt.subplot(4, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])